### Download VIC Parl Ctte Submission PDFs and merge
* V0.1
* Jake Clarke, 24 July 2017

### libraries

In [113]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import wget
import os
from os import listdir
from os.path import isfile, join
from datetime import *

### get web data

In [4]:
# setttings

url = 'https://www.parliament.vic.gov.au/fireservicesbill/inquiries/article/2571'
path = 'https://www.parliament.vic.gov.au'
outputpath = '_tmp_submissions'

print(url)
print(path)
print(outputpath)

https://www.parliament.vic.gov.au/fireservicesbill/inquiries/article/2571
https://www.parliament.vic.gov.au
_tmp_submissions


In [5]:
# get the page
headers = {'User-Agent': UserAgent().chrome}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html5lib')

In [23]:
# for a given <li>, return url
def get_url(block, path):
    link = block.find('a',{'class':'wf_file'})
    file_path = link['href']
    downloadurl = path + file_path
    return downloadurl

# for a given url, download, and rename file
def get_file(url):
    test = wget.download(url, out=outputpath)

In [179]:
# find first ordered list, loop through all <li>s and get url
submissionlist = soup.find('ol')
# for sub in submissionlist.findAll('li'):
#     data = get_url(sub, path)
#     get_file(data)

sub = submissionlist.find('li')
data = get_url(sub, path)
get_file(data)

sub = sub.find_next('li')
data = get_url(sub, path)
get_file(data)

sub = sub.find_next('li')
data = get_url(sub, path)
get_file(data)

sub = sub.find_next('li')
data = get_url(sub, path)
get_file(data)

sub = sub.find_next('li')
data = get_url(sub, path)
get_file(data)

In [33]:
# get list of files
files = [f for f in listdir(outputpath) if isfile(join(outputpath, f))]
print(files[:10])

['S1-Anne_Laver.pdf', 'S2-CONFIDENTIAL.pdf']


In [171]:
## function - make watermark pdf
def make_watermark_pdf(fname,fwatermarkname):
    
    c = canvas.Canvas(fwatermarkname)
    c.setFont("Helvetica", 8)

    text = 'file: ' + str(fname)
    c.drawString(20,60,text)

    ts = datetime.now()
    ts_str = 'downloaded: ' + str(ts)
    c.drawString(20,50,ts_str)

    c.save()

In [173]:
from reportlab.pdfgen import canvas

make_watermark_pdf(
    fname = 'S303-David_Kelly.pdf',
    fwatermarkname = '_tmp_submissions\watermark.pdf'
)

In [164]:
# function -- merge pdf
def merge_with_watermark(fname, foutputname, fwatermarkname):
    output = PdfFileWriter()

    ipdf = PdfFileReader(open(fname, 'rb'))
    wpdf = PdfFileReader(open(fwatermarkname, 'rb'))
    watermark = wpdf.getPage(0)

    for i in xrange(ipdf.getNumPages()):
        page = ipdf.getPage(i)
        page.mergePage(watermark)
        output.addPage(page)

    with open(foutputname, 'wb') as f:
        output.write(f)

In [166]:
from pdfrw import PdfReader, PdfWriter, PageMerge

merge_with_watermark(
    fname = '_tmp_submissions\S1-Anne_Laver.pdf',
    foutputname = '_tmp_submissions\merge\S1-Anne_Laver.pdf',
    fwatermarkname = '_tmp_submissions\watermark.pdf'
)

In [182]:
# for file in files, make watermark, merge into the output dir
fwatermarkname = '_tmp_submissions\watermark.pdf'

files = [f for f in listdir(outputpath) if isfile(join(outputpath, f))]

for file in files:

    fname = '_tmp_submissions\\' + file
    foutputname = '_tmp_submissions\\merge\\' + file
    
    # mkae watermark file containing date stamp and file name
    
    make_watermark_pdf(
        fname,
        fwatermarkname
    )
    
    # merge with watermark file
    
    merge_with_watermark(
        fname,
        foutputname,
        fwatermarkname
    )

In [167]:
from pdfrw import PdfReader, PdfWriter, IndirectPdfDict, PageMerge

In [183]:
# merge files from merge subdir

writer = PdfWriter()

for file in files:
    fname = '_tmp_submissions\\merge\\' + file
    writer.addpages(PdfReader(fname).pages)

writer.write('_tmp_submissions\merge.pdf')